In [1]:
import os, sys
from ZillowHouseData.logger import logger
from ZillowHouseData.exception import CustomException
import pandas as pd
import pickle

In [13]:
class DataPreprocessing:
    def __init__(self, file_name='ZILLOW_DATA_962c837a6ccefddddf190101e0bafdaf.csv', start_date='2012-01-01'):
        #self.file_name = file_name
        #self.file_name = os.path.join('../','artifacts', 'data_ingestion',file_name)
        self.file_name = os.path.join('..','artifacts', 'data_ingestion',file_name)
        self.start_date = start_date

    def data_preprocessing(self):
        dtypes = {
            'indicator_id': 'object',
            'region_id': 'int32',
            'value': 'float32',
            'date': 'object'
        }
        df_list = []
        count = 0

        #raw_data_path: str=os.path.join('artifacts/data_ingestion/','ZILLOW_DATA_962c837a6ccefddddf190101e0bafdaf.csv')
        print("\n printing os path current directory")
        print(os.getcwd())


        file_path = os.path.join('artifacts', 'data_ingestion', 'ZILLOW_DATA_962c837a6ccefddddf190101e0bafdaf.csv')
        print("\n printing file existance")
        print(os.path.exists(file_path))

        
        for chunk in pd.read_csv(self.file_name, chunksize=1000000,
                                 usecols=['indicator_id', 'region_id', 'date', 'value'],
                                 dtype=dtypes,
                                 parse_dates=['date']):
            filtered_chunk = chunk[(chunk['date'] >= self.start_date)]
            df_list.append(filtered_chunk)

        trimmed_df = pd.concat(df_list, ignore_index=True)

        del df_list
        
        trimmed_data = pickle.dumps(trimmed_df)
    
        return trimmed_data

    def get_year_month(self, data):
        trimmed_df = pickle.loads(data)
        trimmed_df['year'] = pd.to_datetime(trimmed_df['date']).dt.year
        trimmed_df['month'] = pd.to_datetime(trimmed_df['date']).dt.month
        trimmed_data = pickle.dumps(trimmed_df)
        return trimmed_data

    def get_stats(self, data):
        trimmed_df = pickle.loads(data)
        intersted_indicators_stats = ['IRAM', 'CRAM', 'MRAM', 'LRAM', 'NRAM', 'SRAM']
        stat_df = trimmed_df[trimmed_df['indicator_id'].isin(intersted_indicators_stats)]
        stat_pivot_df = stat_df.pivot_table(index=['region_id', 'year', 'month'], columns='indicator_id',
                                           values='value', aggfunc='mean')
        stat_pivot_df.reset_index(inplace=True)
        stat_pivot_df.dropna(inplace=True)
        stat_pivot_df.to_csv("stat.csv")
        del stat_df
        trimmed_data = pickle.dumps(trimmed_df)
        return trimmed_data

    def get_merge(self, data):
        trimmed_df = pickle.loads(data)
        df = pd.read_csv('stat.csv')
        intersted_indicators_ZHVI = ['ZATT', 'ZSFH', 'ZALL', 'ZCON', 'ZABT', 'Z2BR', 'Z5BR', 'Z3BR', 'Z1BR', 'Z4BR']
        ZHVI_df = trimmed_df[trimmed_df['indicator_id'].isin(intersted_indicators_ZHVI)]
        del trimmed_df
        final_df = pd.merge(ZHVI_df, df, on=['region_id', 'year', 'month'], how='inner')
        final_data = pickle.dumps(final_df)
        final_df.to_csv("final.csv")
        return final_data


In [7]:
from ZillowHouseData.components.data_preprocessing import DataPreprocessing
from ZillowHouseData.logger import logger
from ZillowHouseData.exception import CustomException
from ZillowHouseData.config.configuration import ConfigurationManager
from ZillowHouseData.pipeline.stage_01_data_ingestion import ingestion_stage
import sys

In [14]:
STAGE_NAME = "Data Preprocessing stage"

class DataPreprocessingTrainingPipeline:
    def __init__(self):
        pass

    def main(self):
            # Create an instance of the DataPreprocessing class
            logger.info(f">>>>>> stage {STAGE_NAME} initated <<<<<<\n\nx==========x")
            data_preprocessor = DataPreprocessing()
            # Replace 'your_data_bytes_here' with the actual data bytes you want to process
            #data = b'your_data_bytes_here'
            # Stage 1: Data Preprocessing
            processed_data = data_preprocessor.data_preprocessing()
            # Stage 2: Extract Year and Month
            logger.info(f">>>>>> exact year & month <<<<<<\n\nx==========x")
            processed_data = data_preprocessor.get_year_month(processed_data)
            # Stage 3: Get Stats
            logger.info(f">>>>>> get stats <<<<<<\n\nx==========x")
            processed_data = data_preprocessor.get_stats(processed_data)
            # Stage 4: Merge Data
            logger.info(f">>>>>> merging data <<<<<<\n\nx==========x")
            final_data = data_preprocessor.get_merge(processed_data)
            print(final_data)



if __name__ == '__main__':
    #ingestion_stage()
    try:
        logger.info(f">>>>>> stage {STAGE_NAME} initated <<<<<<\n\nx==========x")
        obj1 = DataPreprocessingTrainingPipeline()
        obj1.main()
        logger.info(f">>>>>> stage {STAGE_NAME} completed <<<<<<\n\nx==========x")
    except Exception as e:
        logger.exception(e)
        raise CustomException(e,sys)
    


[2023-11-10 20:32:28,322: INFO: 1457263588: >>>>>> stage Data Preprocessing stage initated <<<<<<

x==========x]
[2023-11-10 20:32:28,323: INFO: 1457263588: >>>>>> stage Data Preprocessing stage initated <<<<<<

x==========x]

 printing os path current directory
/Users/keshavkumarelankovan/Desktop/ML_OPS/Skirmish/Zillow-data-ML-project/notebook

 printing file existance
False
[2023-11-10 20:32:28,325: ERROR: 1457263588: [Errno 2] No such file or directory: '. /artifacts/data_ingestion/ZILLOW_DATA_962c837a6ccefddddf190101e0bafdaf.csv']
Traceback (most recent call last):
  File "/var/folders/kp/6197l28d6yzb5wygdgr1hb7w0000gn/T/ipykernel_34285/1457263588.py", line 33, in <module>
    obj1.main()
  File "/var/folders/kp/6197l28d6yzb5wygdgr1hb7w0000gn/T/ipykernel_34285/1457263588.py", line 14, in main
    processed_data = data_preprocessor.data_preprocessing()
  File "/var/folders/kp/6197l28d6yzb5wygdgr1hb7w0000gn/T/ipykernel_34285/324563388.py", line 28, in data_preprocessing
    for chunk

CustomException: Error occured in python script name [/var/folders/kp/6197l28d6yzb5wygdgr1hb7w0000gn/T/ipykernel_34285/1457263588.py] line number [33] error message [[Errno 2] No such file or directory: '. /artifacts/data_ingestion/ZILLOW_DATA_962c837a6ccefddddf190101e0bafdaf.csv']

In [23]:
file_name = 'ZILLOW_DATA_962c837a6ccefddddf190101e0bafdaf.csv'
file = os.path.join('..','artifacts', 'data_ingestion',file_name)
print(os.path.exists(file))
print(os.getcwd())

True
/Users/keshavkumarelankovan/Desktop/ML_OPS/Skirmish/Zillow-data-ML-project/notebook


In [22]:
file_name = 'ZILLOW_DATA_962c837a6ccefddddf190101e0bafdaf.csv'
#script_dir = os.path.dirname(os.path.abspath(__file__))
script_dir = os.path.dirname(os.path.realpath(sys.argv[0]))
print(script_dir)
file_relative_path = os.path.join(script_dir, '..','artifacts', 'data_ingestion', file_name)

# Check if the file exists
print("File exists:", os.path.exists(file_relative_path))

# Print the current working directory
print("Current working directory:", os.getcwd())


/Users/keshavkumarelankovan/Desktop/ML_OPS/Skirmish/Zillow-data-ML-project/mlops_zillow/lib/python3.10/site-packages
File exists: False
Current working directory: /Users/keshavkumarelankovan/Desktop/ML_OPS/Skirmish/Zillow-data-ML-project/notebook


In [25]:
#os.chdir('../')